# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [2]:
import xgboost as xgb
import numpy as np
import pandas as pd
import pickle
import string

class HangmanXGBoostPredictor:
    def __init__(self, model_path="xgboost_hangman_models2.pkl", pos_features=65):
        self.alphabet = list(string.ascii_lowercase)
        self.num_classes = len(self.alphabet)
        self.models = []
        self.pos_features = pos_features
        self._load_models(model_path)

    def _load_models(self, model_path):
        with open(model_path, 'rb') as f:
            self.models = pickle.load(f)

    def _encode_word_state(self, word: str, guessed_letters: set) -> np.ndarray:
        feat = np.full(self.pos_features, -1, dtype=np.int8)
        length = len(word)
        offset = self.pos_features - length

        for i, ch in enumerate(word):
            code = 0 if ch == '*' else ord(ch) - ord('a') + 1
            feat[i] = code
            feat[offset + i] = code

        return feat.reshape(1, -1)

    def predict_letter(self, word: str, guessed_letters: set) -> str:
        features = self._encode_word_state(word, guessed_letters)
        scores = []
        # print(self.models)
        for idx, model in enumerate(self.models.items()):
            # print(idx,model)
            model=model[1]
            letter = self.alphabet[idx]
            if letter in guessed_letters:
                scores.append(-np.inf)
                continue
            prob = model.predict_proba(features)[0][1]  # probability of presence
            scores.append(prob)

        return scores


In [17]:
import torch
from transformers import CanineTokenizer, CanineConfig, CanineForSequenceClassification
from transformers import CanineConfig, CanineTokenizer, AutoModelForSequenceClassification

# # Constants for separating and masking in the CANINE input sequence
# CANINE_SEP_TOKEN   = " [SEP] "
# CANINE_MASK_TOKEN  = "[MASK]"

class CanineHangmanPlayer:
    def __init__(self, pretrained_model_path: str = "google/canine-s", device: torch.device = None,xgb=None):
        # Load CANINE tokenizer & config
        self.tokenizer = CanineTokenizer.from_pretrained('google/canine-s')
        self.config    = CanineConfig.from_pretrained(pretrained_model_path)
        self.config.num_labels = 26
        self.xgb=xgb
        self.nice=1
        self.trigger=1
        self.all_letters = [chr(i) for i in range(ord('a'), ord('z')+1)]
        self.state=False
        self.guesses={}

        
        # Set up device
        self.device = "cuda"#device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # Load a sequence-classification head on top of CANINE
        self.model =AutoModelForSequenceClassification.from_pretrained(
            pretrained_model_path,
            config=self.config).to(self.device)
        
        # Tokens for building the game state string
        self.CANINE_MASK_TOKEN = self.tokenizer.mask_token
        self.CANINE_SEP_TOKEN = self.tokenizer.sep_token
        
        # Toggle for self-play finetuning
        self.training = False

    def eval(self):
        self.model.eval()

    def simulate_hangman_transformers(self, word: str, max_wrong_guesses: int = 6, verbose: int = 1):
        """
        Play hangman against the CANINE model.
        If self.training is True, returns (model_logits_seq, true_dist_seq, success_flag).
        Otherwise returns just susccess_flag.
        """
        # Build index map of letters→positions
        word_idxs = {}
        for i, c in enumerate(word):
            word_idxs.setdefault(c, []).append(i)

        all_letters = [chr(i) for i in range(ord('a'), ord('z')+1)]
        guesses      = {}
        encoded_word = "*" * len(word)
        num_wrong= 0

        self.eval()
        if self.training:
            outputs_model = []
            outputs_true  = []

        # if verbose:
        #     print(f"[WORD]: {word}")


        while encoded_word != word and num_wrong < max_wrong_guesses:
            missing_count = encoded_word.count("*")

            # if missing_count <= 0:
            #     masked = encoded_word.replace("*", "0")
            #     guess = self.lstm_model.predict(masked, guessed_letters=set(guesses.keys()))
            #     print(guess,end=" ")
            # else:
            
            check = self.xgb.predict_letter(''.join(encoded_word), guesses)
            state = ''.join(guesses.keys()) + self.CANINE_SEP_TOKEN + encoded_word.replace('*', self.CANINE_MASK_TOKEN)
            enc = self.tokenizer(state, padding="max_length", truncation=True, max_length=64, return_tensors="pt").to(self.device)

            with torch.no_grad():
                logits = self.model(**enc).logits

            arr = logits.cpu().numpy()[0]
            arr1=np.argsort(arr)[::-1].tolist()
            total = sum(arr)
            guess_idx = np.argmax(arr)
            arr = [x / total for x in arr]
            arr1=np.argsort(check)[::-1].tolist()
            rank1=np.argsort(check)[::-1].tolist()
            pos1=rank1.index(guess_idx)
            pos2=all_letters[guess_idx]
            # guess = all_letters[guess_idx]
            org=arr[guess_idx]
            verify=check[guess_idx]
            return rank1[0]
            # trigger=0
            trigger=False
            if org<0.002 and pos1>=7 and missing_count<=4:
                # count+=1
                # change=1
                # print("Trigger")
                trigger=True
                # self.trigger+=1
                guess_idx = rank1[0]
            guess = all_letters[guess_idx]


            if not trigger:
                while guess in guesses:
                    arr[guess_idx] = -np.inf 
                    guess_idx = np.argmax(arr)
                    guess = all_letters[guess_idx]
            
            if pos2!=guess and pos2 not in word_idxs and guess not in word_idxs:
                self.trigger-=1
            # Apply the guess
            if guess in word_idxs:
                self.nice+=1 if trigger else 0
                # self.trigger+=1
                for pos in word_idxs[guess]:
                    encoded_word = encoded_word[:pos] + guess + encoded_word[pos+1:]
            else:
                self.trigger+=1 if trigger else 0
                num_wrong += 1
            trigger=False

            guesses[guess] = True
            if verbose == 1:
                print(f"  Guess: {guess.upper():<2} → {encoded_word}  (Wrong: {num_wrong}) {verify:.3f} {org:.3f} {pos1}, 'Org', {pos2} {all_letters[rank1[0]]} ")

        success = (encoded_word == word)
        if verbose == 3 or verbose==1:
            print(f"Result: {'✅ CORRECT' if success else '❌ FAILED'} | Final: {encoded_word} {word}  \n")
        # print(nice/count*100)
        if self.training:
            return torch.vstack(outputs_model), torch.vstack(outputs_true), success
        return success
    
    def test_accuracy(self, words, verbose=1):
        # global nice,trigger
        # nice,trigger=1,1
        n=len(words)
        count=1
        correct = 0
        for w in words:
            print(f"{(correct / count) * 100:.2f} {count} / {n} {self.nice} {self.trigger} ",end="\r")
            correct += self.simulate_hangman_transformers(w, verbose=verbose)
            count+=1
        return correct / len(words)
    def predict(self,word,guesses):

            encoded_word=word.replace(" ","").replace("_","*")
            missing_count = encoded_word.count("*")

            guesses={x:True for x in guesses}
                

            check = self.xgb.predict_letter(''.join(encoded_word), guesses)
            state = ''.join(guesses.keys()) + self.CANINE_SEP_TOKEN + encoded_word.replace('*', self.CANINE_MASK_TOKEN)
            enc = self.tokenizer(state, padding="max_length", truncation=True, max_length=64, return_tensors="pt").to(self.device)

            with torch.no_grad():
                logits = self.model(**enc).logits

            arr = logits.cpu().numpy()[0]
            arr1=np.argsort(arr)[::-1].tolist()
            
            total = sum(arr)
            guess_idx = np.argmax(arr)
            arr = [x / total for x in arr]
            
            arr1=np.argsort(check)[::-1].tolist()
            rank1=np.argsort(check)[::-1].tolist()
            pos1=rank1.index(guess_idx)
            # guess = all_letters[guess_idx]
            org=arr[guess_idx]
            verify=check[guess_idx]
            return self.all_letters[rank1[0]]


            if org<0.005 and pos1>=7 and missing_count<=3:
                # print("Trigger")
                trigger=True
                guess_idx = rank1[0]
            guess = self.all_letters[guess_idx]
            # guesses[guess]=True
            return guess

        




In [18]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

    
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [19]:
xgb = HangmanXGBoostPredictor("xgboost_hangman_models2.pkl")
model=CanineHangmanPlayer("10epoch",xgb=xgb)
model.training=False



In [20]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################
        ans=model.predict(word,self.guessed_letters)
        return ans
 
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [14]:
api = HangmanAPI(access_token="c84a1770149d9f55d91a25f8b13df6", timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [8]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs

print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Successfully start a new game! Game ID: 5ae6e65b55a2. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': '5ae6e65b55a2', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ e _ _ _ _ _ _ '}
Guessing letter: i
Sever response: {'game_id': '5ae6e65b55a2', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ e _ _ _ _ i _ '}
Guessing letter: o
Sever response: {'game_id': '5ae6e65b55a2', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ e _ _ _ _ i _ '}
Guessing letter: a
Sever response: {'game_id': '5ae6e65b55a2', 'status': 'ongoing', 'tries_remains': 5, 'word': 'a _ a _ e _ _ a _ i _ '}
Guessing letter: c
Sever response: {'game_id': '5ae6e65b55a2', 'status': 'ongoing', 'tries_remains': 5, 'word': 'a _ a _ e _ _ a _ i c '}
Guessing letter: t
Sever response: {'game_id': '5ae6e65b55a2', 'status': 'ongoing', 'tries_remains': 5, 'word': 'a _ a _ e _ _ a t i c '}
Guessing letter: r
Sever response: {'game_id': '5ae6e65b55a2', 

In [21]:
total,s=total_practice_runs,total_practice_successes
for i in range(200):
    api.start_game(practice=1,verbose=True)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = (total_practice_successes-s) / (total_practice_runs-total)*100
    time.sleep(0.5)
    print(f"{practice_success_rate:.2f} {total_practice_successes - s} {total_practice_runs - total}", end="\r")

Successfully start a new game! Game ID: 094877fbcbc5. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': '094877fbcbc5', 'status': 'ongoing', 'tries_remains': 6, 'word': 'e _ e _ _ _ _ _ _ '}
Guessing letter: r
Sever response: {'game_id': '094877fbcbc5', 'status': 'ongoing', 'tries_remains': 6, 'word': 'e _ e _ _ r _ _ _ '}
Guessing letter: i
Sever response: {'game_id': '094877fbcbc5', 'status': 'ongoing', 'tries_remains': 6, 'word': 'e _ e _ _ r i _ _ '}
Guessing letter: n
Sever response: {'game_id': '094877fbcbc5', 'status': 'ongoing', 'tries_remains': 5, 'word': 'e _ e _ _ r i _ _ '}
Guessing letter: s
Sever response: {'game_id': '094877fbcbc5', 'status': 'ongoing', 'tries_remains': 4, 'word': 'e _ e _ _ r i _ _ '}
Guessing letter: t
Sever response: {'game_id': '094877fbcbc5', 'status': 'ongoing', 'tries_remains': 4, 'word': 'e _ e _ t r i _ _ '}
Guessing letter: c
Sever response: {'game_id': '094877fbcbc5', 'status': 'ongoing', 'tries_

KeyboardInterrupt: 

In [22]:
print(total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes)

84 0 0 19


## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [23]:
for i in range(1000):
    print('Playing ', i, ' th game')
    #Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)
    
    #DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Playing  1  th game
Playing  2  th game
Playing  3  th game
Playing  4  th game
Playing  5  th game
Playing  6  th game
Playing  7  th game
Playing  8  th game
Playing  9  th game
Playing  10  th game
Playing  11  th game
Playing  12  th game
Playing  13  th game
Playing  14  th game
Playing  15  th game
Playing  16  th game
Playing  17  th game
Playing  18  th game
Playing  19  th game
Playing  20  th game
Playing  21  th game
Playing  22  th game
Playing  23  th game
Playing  24  th game
Playing  25  th game
Playing  26  th game
Playing  27  th game
Playing  28  th game
Playing  29  th game
Playing  30  th game
Playing  31  th game
Playing  32  th game
Playing  33  th game
Playing  34  th game
Playing  35  th game
Playing  36  th game
Playing  37  th game
Playing  38  th game
Playing  39  th game
Playing  40  th game
Playing  41  th game
Playing  42  th game
Playing  43  th game
Playing  44  th game
Playing  45  th game
Playing  46  th game
Playing  47  th game
Pl

HangmanAPIError: {'error': 'You have reached 1000 of games', 'status': 'denied'}

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [33]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

ZeroDivisionError: division by zero